In [26]:
from sparse_embedding import SparseRetrieval, timer
from transformers import AutoTokenizer
from datasets import load_from_disk, concatenate_datasets
import pandas as pd
from pprint import pprint

pd.set_option('display.max_colwidth', 150)

In [22]:
def dataset_to_df(dataset_name):
    data_list = []

    # 필요한 항목 추출
    for item in dataset_name:
        data_list.append({
            'question': item['question'],
            'context': item['context'],
            'title': item['title'],
            'document_id': item['document_id'],
        })

    # pandas dataframe으로 변환
    result_df = pd.DataFrame(data_list)
    return result_df


def result_to_df(tuple_result):
    retrieved_df = pd.DataFrame(tuple_result).T
    retrieved_df.columns = ['score','retrieved_docs']
    retrieved_df['ranking'] = range(1, len(retrieved_df) + 1)  # ranking 열 추가
    retrieved_df['score'] = retrieved_df['score'].round(3)  # score 열 반올림
    retrieved_df = retrieved_df[['ranking', 'score', 'retrieved_docs']]  # 열 순서 변경: ranking을 맨 앞으로
    return retrieved_df

## 하나의 예시에 대한 retrieval 결과 조회

In [17]:
# 파라미터 정의
tokenizer_model = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, use_fast=True)
evaluation_methods = ['hit', 'mrr']
topk=10

In [18]:
# retriever 준비
retriever_tfidf = SparseRetrieval(
    embedding_method='tfidf',
    tokenizer=tokenizer,
    topk=topk,
    use_faiss=False,
    retrieval_data_path="../../data/wikipedia_documents.json",
    eval_data_path="../../data/train_dataset",
)
retriever_tfidf.load_data()  # 데이터 불러오기
retriever_tfidf.get_sparse_embedding()  # 임베딩 생성/불러오기

retriever_bm25 = SparseRetrieval(
    embedding_method='bm25',
    tokenizer=tokenizer,
    topk=topk,
    use_faiss=False,
    retrieval_data_path="../../data/wikipedia_documents.json",
    eval_data_path="../../data/train_dataset",
)
retriever_bm25.load_data()  # 데이터 불러오기
retriever_bm25.get_sparse_embedding()  # 임베딩 생성/불러오기

In [19]:
# 평가용 데이터셋 준비
org_dataset = load_from_disk("../../data/train_dataset/")
full_ds = concatenate_datasets([org_dataset["train"].flatten_indices(), org_dataset["validation"].flatten_indices()])
org_df = dataset_to_df(full_ds)

### 예시 1: 미쿠를 다른 세계로 이끈 정체는?

In [29]:
# 랜덤 질문 추출
sampled_df = org_df.sample(1, random_state=10)
question = sampled_df['question'].iloc[0]
ground_truth_context = sampled_df['context'].iloc[0]

# retrieve
tfidf_result = retriever_tfidf.retrieve(question, topk)
bm25_result = retriever_bm25.retrieve(question, topk)
tfidf_df = result_to_df(tfidf_result)
bm25_df = result_to_df(bm25_result)

print('-'*100)
print(f"[Question]\n{question}\n")
print("[Ground Truth Context]")
pprint(ground_truth_context)
print('-'*100)
tfidf_df

----------------------------------------------------------------------------------------------------
[Question]
미쿠를 다른 세계로 이끈 정체는?

[Ground Truth Context]
('주인공 아마하시 미쿠는 학생회 서기이자 문예부원. 미쿠는 문화제의 메인 이벤트 연극의 각본을 맡게 된다. 각본 제작을 위해 건네받은 '
 '원작을 읽는 중 미쿠는 신기한 검은 고양이에게 이끌려 학교 중심에 있는 샘을 통해 아라비아 풍의 이세계(異世界)를 헤매게 된다. 그 '
 "나라에는 현실의 학교에서 보았던 학우들이 그 세계의 거주자로 존재하고, 현실 세계와는 다른 '또 하나의 얼굴'을 히로인에게 드러낸다. "
 '그녀는 그것을 꿈이라고 생각했다. 하지만 그 날 뒤로 매일 밤 꾸듯이 되어버린 그 꿈은 왠지 확실한 감촉이 남아있고, 꿈이라고 하기엔 '
 '너무나 리얼했다.\\n\\n그 꿈 속 세계의 이름은 〈미후타후〉. 그녀가 다니는 미후타 학원과 너무나 비슷한 이름의 세계였다. 나라를 '
 '통치하는 왕이 사라지고, 왕과 함께 〈미후타후〉를 유지하고 지키는 무녀도 오랫동안 나타나지 않아 세계는 조용히 쇠퇴하고 있었다. 신성한 '
 '샘에서 나타난 그녀는 5명의 왕자 후보 중에서 왕을 선택하여 〈미후타후〉를 구하는 운명을 지닌 신성한 무녀가 된다. 거울처럼 서로 영향을 '
 '주고 받으며 같은 시간을 반복하는 현실과 이세계(異世界), 시간 마저 잘라내버린 모형정원에서 반복되는 비극. 그것을 피하기 위해 미쿠는 '
 '자신의 손으로 두 세계의 이야기를 다시 바꿔 써 내려간다.')
----------------------------------------------------------------------------------------------------


,ranking,score,retrieved_docs
0,1,0.320024,; 〈악의 딸〉노벨 시리즈 ()\n:악의P(mothy)가 카가미네 린·렌을 이용하여 발표 한 곡〈악의 딸〉을 시작으로 하는 악곡의 소설화. 2010년 8월부터 2012년 3월까지 총 4권이 출간되었다. 하츠네 미쿠를 모델로 한 캐릭터 “미카엘라”가 등장한다.\n...
1,2,0.277526,"; 13세의 헬로 워크 DS (디지털 웍스 엔터테인먼트)\n: 하츠네 미쿠 소비자 소프트 첫 출연이되는 2008년 5월 29일에 발매 된 닌텐도 DS용 게임 소프트. 뮤지션으로 등장하고 주인공 카토 미쿠(加藤ミク, 하츠네 미쿠랑 동명이지만 관련되지 않음)의 직업..."
2,3,0.272049,;마이니치 잇쇼/주간 토로·스테이션 (소니 컴퓨터 엔터테인먼트)\n:플레이스테이션 3 소프트《마이니치 잇쇼》에서 방송된 정보 프로그램 토로·스테이션 제373회 (2007 년 11월 16일)에 디폴메 캐릭터의 “하츄네 미쿠”가 게스트로 출연해 노래도 선보였다 . ...
3,4,0.265546,; 넨도로이드 하츠네 미쿠 (굿 스마일 컴퍼니)\n: 2008년 3월 31일 발매. 데포 피겨. 일반적인 표정 외에 부속품으로 하츄네 미쿠 페이스와 파가 붙어 있다.\n:붐 “주동자”라고도 말할 수 있는 니코니코 동화에서는 2007년 11월 26일부터 자정에서 ...
4,5,0.257644,"; 하츠네 미쿠 -Project DIVA- (세가)\n: 2008년 일본에서 굿 디자인 제품상, 성운상 자유 부문을 수상했으며, 세가에서 하츠네 미쿠를 주인공으로 플레이스테이션 포터블 전용으로 《하츠네 미쿠 -Project DIVA-》라는 제목으로 게임이 제작되..."
5,6,0.250476,"; 하츠네 미쿠 MIXING BOX (코단샤)\n: 2008년 9월 25일 발매. 피아프로에서 공모된 곡이나 일러스트를 수록한 DVD, 팬북, 피겨가 수록되어 있다.\n; 전격 레이야즈 bible Vol.2 VOCALOID (아스키 미디어 웍스)\n: 2010년..."
6,7,0.243449,"2010년 4월 30일, 하츠네 미쿠의 목소리를 확장시킨 어펜드 버전이 발매되었으며, 2013년 8월 31일 VOCALOID3를 채용 한 “하츠네 미쿠 V3 ENGLSIH”가 발매되었으며, 같은 해 9월에는 하츠네 미쿠와 어펜드 중 4개의 라이브러리를 수록한 “..."
7,8,0.237038,"고교 1학년인 이케노우치 미쿠는 사이가 좋은 여자 친구(후미코 등)도 연상의 남자친구(카타오카 선배)도 있어, 근심하는 것은 전혀 없었다. 그러나 돌연 아버지가 하숙을 시작한다고 말해, 미쿠의 생활은 완전히 바뀐다. 그리고, 하숙하는 사람 중 한 명인 나라하시 ..."
8,9,0.210628,"보컬은 음성 합성 소프트 하츠네 미쿠를 이용하고 있다. 초회 한정반은 슬리브 사양이 되어 있어, 자켓 일러스트를 우키 아츠야가 담당했다.\n\n「적란운 그라피티」는 「ryo (supercell) feat.하츠네 미쿠」명의, 「Fallin' Fallin' Fall..."
9,10,0.208659,주인공 아마하시 미쿠는 학생회 서기이자 문예부원. 미쿠는 문화제의 메인 이벤트 연극의 각본을 맡게 된다. 각본 제작을 위해 건네받은 원작을 읽는 중 미쿠는 신기한 검은 고양이에게 이끌려 학교 중심에 있는 샘을 통해 아라비아 풍의 이세계(異世界)를 헤매게 된다. ...


In [30]:
bm25_df

,ranking,score,retrieved_docs
0,1,26.401389,뉴욕 글로브 신문사의 사장은 존스를 2차세계대전의 전운이 감도는 유럽에 파견하기로 낙점한다. 존스는 암스테르담으로 가 외교관 밴 미어를 인터뷰하라는 첫 임무를 지시받게 된다.\n\n존스는 피셔가 당수로 있는 세계평화당의 모임에 밴 미어가 참석할 거라는 정보를 사...
1,2,25.870688,주인공 아마하시 미쿠는 학생회 서기이자 문예부원. 미쿠는 문화제의 메인 이벤트 연극의 각본을 맡게 된다. 각본 제작을 위해 건네받은 원작을 읽는 중 미쿠는 신기한 검은 고양이에게 이끌려 학교 중심에 있는 샘을 통해 아라비아 풍의 이세계(異世界)를 헤매게 된다. ...
2,3,25.465193,;와다 아키라(和田 晶|와다 아키라)\n:그녀의 친구 아야메와 사치와 함께 다시 시작된 만화에서 등장하는 새로운 등장 인물이다. 밴드에서 온나구미(온나구미) 라고 불린다. 그녀는 짧은 검정머리를 가지고 있고 그녀의 머리가 지저분할 때마다 협박하는 것처럼 보인다...
3,4,25.370926,"리차르도 미누톨로는 유부남으로 다른 유부녀를 열렬히 사랑하여 구애하지만 이 유부녀가 워낙 제 남편을 좋아하여 거절만 당한다. 리차르도 미누톨로는 꾀를 써서 이제 유부녀를 깨끗이 포기하고 다른 여자를 사랑하는 척 하면서, 유부녀에게 그냥 지나가는 말로 그 질투심을..."
4,5,25.214899,"요사쿠\n\n:성우:?\n:성우:이상범\n'피범벅 요사쿠'로 통한다.'노킹마스터'지로의 손자인 텟페이의 스승이며 세계 최고의 재생가 중 한명이다.토리코가 팔을 잃었을때 다시 재생시켜 주었고,그 외 토리코 일행에게 여러가지 도움을 주었다. 실력은 확실. 섬세함이 ..."
5,6,23.798586,"현용수(Yong-Soo Hyun) 박사는 4000년 동안 성공한 민족으로 검증된 유대인 교육을 연구하는 쉐마교육연구원을 설립한 학자이자 교육자다. 그는 1996년에 펴낸 유대인 교육서, 《IQ는 아버지 EQ는 어머니 몫이다》가 자녀 교육 분야의 필독서로 자리 잡..."
6,7,23.705285,"슌텐(舜天, 1166년? ~ 1237년?)은 슌텐 왕통(舜天王統)의 개조(開祖)로 여겨지는 류큐왕국의 국왕(재위: 1187년 ~ 1237년)이다. 신호(神號)는 미코토 아쓰시(尊敦)로, 실재를 증명할 동시대 사료가 존재하지 않고 있으나, 그 이름은 16세기경부터..."
7,8,23.584788,"1940년 이온 안토네스쿠는 쿠데타로 카롤 2세를 몰아내고, 미하이 1세를 즉위시켰다. 총리가 된 안토네스쿠는 파시스트 체제를 지향하여 루마니아 공산당을 불법 정당으로 탄압하고, 전 국민을 전쟁터로 내몰았다. 제2차 세계대전으로 루마니아 민중들이 고통을 받은 이..."
8,9,23.213845,탈세계화라는 용어는 경제 활동의 일부로서 등장했다. 특히 1914 년과 1970 년대 사이에 무역지수가 하락한 많은 선진국에서 나타난 매우 심오한 변화 중 하나이다. 무역지수의 감소는 경제 세계화가 많은 범위에서 심화되고 있음에도 불구하고 그들의 경제는 다른 ...
9,10,23.169605,;카가미\n:성우: 후루사와 토오루\n:샌들과 백의가 트레이드 마크인 양호실 선생님. 항상 마이페이스로 남의 얘기를 전혀 듣지 않는 것처럼 보이지만 때때로 던져주는 어드바이스는 정확하며 신랄(?)하다. 미쿠에 대해서도 호의적인지 악의적인지 판단하기 힘든 태도를 ...
